This walkthrough-style Jupyter notebook will detail the methodology used to evaluate the predictive performance of the underlying forecasting models in the Residential Energy Demand Forecasting Pipeline.

In [1]:
import pandas as pd
import pickle as pkl
import torch as t
from scipy.stats import wilcoxon
import plotly.graph_objects as go


Fit Forecasting Model on all training data

Fit or load Prophet-VAR model 

In [ ]:
if True & ():
    pv_

In [ ]:
# save forecasting model?

Load Saved Forecasting Model from repository 

Sensitivity Analysis